<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Modelisation_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import of the training data

from google.colab import files
uploaded = files.upload()

# Import of moduls

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier



from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading the data
train_df = pd.read_json('train_df_complete.json', orient="records", lines=True)
dev_df = pd.read_json('dev_df_complete.json', orient="records", lines=True)

Saving dev_df_complete.json to dev_df_complete.json
Saving train_df_complete.json to train_df_complete.json


In [2]:
# Get the list of explicatives variables
list_var = ['subclass_depth', 'subclasses', 'instances_of', 'instances_of_up', 'date', 'nb_lang',
       'descr_num_nouns', 'descr_num_verbs',
       'descr_num_adjectives', 'descr_has_location', 'descr_has_ethnic_group',
       'descr_has_event', 'type_cat', 'category_architecture',
       'category_biology', 'category_comics and anime', 'category_fashion',
       'category_films', 'category_food', 'category_geography',
       'category_literature', 'category_media', 'category_music',
       'category_politics', 'category_sports', 'category_transportation',
       'category_visual arts', 'category_combined',
       'main_country_cat'
]

# Create the X and y, train and dev
X_train = train_df[list_var]
y_train = train_df['label_int'].values

X_dev = dev_df[list_var]
y_dev = dev_df['label_int'].values

# Select numerical columns from X_train (all of them are int64 or float64)
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Define pipeline: impute Nan, then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Apply pipeline to numeric columns
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols)
])

# Fit on X_train and transform it on X_train and X_dev
X_train_processed = preprocessor.fit_transform(X_train)
X_dev_processed = preprocessor.transform(X_dev)

# COnvertion
X_train_processed = X_train_processed.astype('float32')
X_dev_processed = X_dev_processed.astype('float32')

# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=3)
y_dev_cat = to_categorical(y_dev, num_classes=3)




# Model


# Callbacks
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Model
model = models.Sequential()
model.add(layers.Input(shape=(29,)))

for units in [1024, 1024, 512, 512, 256, 256, 128, 128, 64, 64, 32, 32]:
    model.add(layers.Dense(units))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3 if units > 128 else 0.2))

model.add(layers.Dense(3, activation='softmax'))

# Compilation
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

history = model.fit(
    X_train_processed, y_train_cat,
    validation_data=(X_dev_processed, y_dev_cat),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1024)           │        30,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 2,146,083 (8.19 MB)

 Trainable params: 2,138,019 (8.16 MB)

 Non-trainable params: 8,064 (31.50 KB)

Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 138ms/step - accuracy: 0.3901 - loss: 1.4207 - val_accuracy: 0.4200 - val_loss: 1.0263 - learning_rate: 0.0010
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - accuracy: 0.5796 - loss: 0.9515 - val_accuracy: 0.5733 - val_loss: 0.7919 - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6421 - loss: 0.8125 - val_accuracy: 0.6967 - val_loss: 0.6867 - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6784 - loss: 0.7556 - val_accuracy: 0.7133 - val_loss: 0.6804 - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6976 - loss: 0.7163 - val_accuracy: 0.7400 - val_loss: 0.6323 - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7026 - loss: 0.7063 - val_accuracy: 0.7000 - val_loss: 0.6505 - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7052 - loss: 0.7001 -

In [3]:
# Predictions on train
y_train_pred = model.predict(X_train_processed)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(y_train_cat, axis=1)

# Predictions on dev
y_dev_pred = model.predict(X_dev_processed)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(y_dev_cat, axis=1)

# Accuracy
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)

print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")


196/196 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step
Train accuracy : 77.75%
Dev accuracy   : 73.00%


# Random Forest

In [4]:
# From one hot to labels
y_train_int = np.argmax(y_train_cat, axis=1)
y_dev_int = np.argmax(y_dev_cat, axis=1)

# Random Forest model, and training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_processed, y_train_int)

# Predictions
y_train_pred = model.predict(X_train_processed)
y_dev_pred = model.predict(X_dev_processed)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_int, y_train_pred)
dev_accuracy = accuracy_score(y_dev_int, y_dev_pred)

print(f"Accuracy on Train: {train_accuracy * 100:.2f}%")
print(f"Accuracy on Dev: {dev_accuracy * 100:.2f}%")


Accuracy on Train: 99.70%
Accuracy on Dev: 75.00%


# XGBoost

In [5]:
# Define XGBoost model
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Train model
model.fit(X_train_processed, y_train)

# Predictions
y_train_pred = model.predict(X_train_processed)
y_dev_pred = model.predict(X_dev_processed)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
dev_accuracy = accuracy_score(y_dev, y_dev_pred)

print(f"Accuracy on Train: {train_accuracy * 100:.2f}%")
print(f"Accuracy on Dev: {dev_accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:17:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy on Train: 90.91%
Accuracy on Dev: 72.33%


# SVM

In [6]:
# From one hot to labels
y_train_int = np.argmax(y_train_cat, axis=1)
y_dev_int = np.argmax(y_dev_cat, axis=1)

# Model SVM and training
model = SVC(kernel='rbf', probability=True)
model.fit(X_train_processed, y_train_int)

# Predictions
y_train_pred = model.predict(X_train_processed)
y_dev_pred = model.predict(X_dev_processed)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_int, y_train_pred)
dev_accuracy = accuracy_score(y_dev_int, y_dev_pred)

print(f"Accuracy on Train: {train_accuracy * 100:.2f}%")
print(f"Accuracy on Dev: {dev_accuracy * 100:.2f}%")

Accuracy on Train: 77.75%
Accuracy on Dev: 74.00%


# Multimodal regression

In [7]:
# From one hot to labels
y_train_int = np.argmax(y_train_cat, axis=1)
y_dev_int = np.argmax(y_dev_cat, axis=1)

# Model : Logistic regression. Training
model = LogisticRegression(multi_class='multinomial', max_iter=1000)
model.fit(X_train_processed, y_train_int)

# Prediction
y_train_pred = model.predict(X_train_processed)
y_dev_pred = model.predict(X_dev_processed)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_int, y_train_pred)
dev_accuracy = accuracy_score(y_dev_int, y_dev_pred)

print(f"Accuracy on Train: {train_accuracy * 100:.2f}%")
print(f"Accuracy on Dev: {dev_accuracy * 100:.2f}%")

Accuracy on Train: 75.04%
Accuracy on Dev: 76.33%
